## Set a double strategy. Train TOP LAYER + FULL MODEL. REGULARIZATION

In [12]:
import os
import numpy as np

from silence_tensorflow import silence_tensorflow
silence_tensorflow()  # silence TF warnings
import tensorflow as tf

from dataProcessing import load_file, encode_data, insert_target, processingScriber00
from transformers import AutoTokenizer
from transformers import TFCamembertForMaskedLM
from datetime import datetime
import json
import sys

In [24]:
### instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained("jplu/tf-camembert-base", do_lower_case=True)

### punctuation encoder
punctuation_enc = {
    'O': 0,
    'PERIOD': 1,
}

### hyper-parameters
vocabSize = 32005
sequenceSize = 11
batchSize = 32
learat = 1e-5
trainLayerInd = 0
training = True
droPro = 0.4
numEpoTop = 3
numEpoAll = 8

listHyper0 = ['vocabSize', 'sequenceSize', 'batchSize', 'learat', 'trainLayerInd',
              'training', 'dropoutProbability', 'numEpoTop', 'numEpoAll']
listHyper1 = [str(vocabSize), str(sequenceSize), str(batchSize), str(learat), str(trainLayerInd),
              str(training), str(droPro), str(numEpoTop), str(numEpoAll)]
time = datetime.now().strftime("%Y%m%d_%H%M%S")
save_path = 'ModelsExpScriber/{}/'.format(time)
os.mkdir(save_path)

### Process Data

In [25]:
print('\nPROCESS DATA')

# name of dataset with sentences
data_name = "Scriber"

# file name
trainDataName = 'Data' + data_name + '/' + 'raw.processed.Train_01.txt'

# from sentences to list of words+punctuation
data_train = load_file(processingScriber00(trainDataName))

# encode data and insert target
X_train_, y_train_ = encode_data(data_train, tokenizer, punctuation_enc)
X_train = insert_target(X_train_, sequenceSize)
y_train = np.asarray(y_train_)

# ### get only a fraction of data 
# n = 32
# X_train = X_train[0:n]
# y_train = y_train[0:n]

# build the datasets
trainDataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=500000).batch(batchSize)


PROCESS DATA


In [26]:
X_train.shape

(1821205, 11)

In [27]:
X_train[0, :]

array([  76,   11,   73,   23,   52,    0,  427, 5061, 5819,   36,  366])

In [28]:
lst = [44,   27,   76,   11,  524, 3007,  514,   79,  484,  202, 76,
       11,   73,   23,   52,    0,  427, 5061, 5819,   36,  366, 1065,
       773, 5061, 5615,   50,   39, 2668,  398,   27,   76,   11]
tokenizer.convert_ids_to_tokens(lst)

['▁ce',
 '▁que',
 '▁j',
 "'",
 'avais',
 '▁raconte',
 '▁maintenant',
 '▁comme',
 '▁quoi',
 '▁moi',
 '▁j',
 "'",
 'ai',
 '▁un',
 '▁s',
 '<s>NOTUSED',
 'fr',
 '▁bonjour',
 '▁bienvenue',
 '▁au',
 '▁service',
 '▁client',
 '▁oui',
 '▁bonjour',
 '▁monsieur',
 '▁je',
 '▁vous',
 '▁appelle',
 '▁parce',
 '▁que',
 '▁j',
 "'"]

### Build Model

In [ ]:
print('\nBUILD THE MODEL, WITH REGULARISATION')

### get configuration file
modelBERT = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")
configBERT = modelBERT.config

### change dropout probability
configBERT.hidden_dropout_prob = droPro

bert_input = tf.keras.Input(shape=(sequenceSize), dtype='int32', name='bert_input')
x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base", config=configBERT)(bert_input, training=training)[0]
x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
dense_out = tf.keras.layers.Dense(2)(x)

model = tf.keras.Model(bert_input, dense_out, name='model')

# define the loss
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
def loss(model, x, y):
    y_ = model(x)
    return loss_object(y_true=y, y_pred=y_)

# func to calculate the gradients
def grad(model, inputs, targets, trainLayerInd):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return loss_value, tape.gradient(loss_value, model.trainable_variables[trainLayerInd:])

# define the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learat)

### Training loop. TOP LAYER

In [ ]:
print("\nEXPERIMENT FOLDER: ", time)

print("\nHYPERPARAMETERS")
print("Sequence Size = ", sequenceSize)
print("Batch Size = ", batchSize)
print("numEpoTop = ", numEpoTop)
print("numEpoAll = ", numEpoAll)

print("\nTRAINING DATASET TENSOR SHAPE = ", X_train.shape)

print("\nTRAINING TOP LAYER")

trainLayerInd = -2  # top layer only

epoch_loss_avg = tf.keras.metrics.Mean()
epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

train_loss_results = []
train_accuracy_results = []

checkpoint_path = save_path + "cp-{epoch:03d}.ckpt"

tmpTrain = np.inf
for epoch in range(1, numEpoTop+1):

    # training loop
    for x, y in trainDataset:
        # optimize the model
        loss_value, grads = grad(model, x, y, trainLayerInd)
        optimizer.apply_gradients(zip(grads, model.trainable_variables[trainLayerInd:]))

        # track progress
        epoch_loss_avg.update_state(loss_value)
        epoch_accuracy.update_state(y, model(x))

    # end epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    print("\nEpoch {:03d}: (Training)   Loss: {:.3f}, Accuracy: {:.3%}".format(epoch, epoch_loss_avg.result(), epoch_accuracy.result()))

    # # save model if new min for train loss is found
    # if epoch_loss_avg.result().numpy() < tmpTrain:
    tmpTrain = epoch_loss_avg.result().numpy()
    model.save_weights(checkpoint_path.format(epoch=epoch))

    epoch_loss_avg.reset_states()
    epoch_accuracy.reset_states()

### Training loop. FULL MODEL

In [ ]:
print("\nTRAINING FULL MODEL")

trainLayerInd = 0  # full model

tmpTrain = np.inf
for epoch in range(numEpoTop+1, numEpoTop+numEpoAll+1):

    # training loop
    for x, y in trainDataset:
        # optimize the model
        loss_value, grads = grad(model, x, y, trainLayerInd)
        optimizer.apply_gradients(zip(grads, model.trainable_variables[trainLayerInd:]))

        # track progress
        epoch_loss_avg.update_state(loss_value)
        epoch_accuracy.update_state(y, model(x))

    # end epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    print("\nEpoch {:03d}: (Training)   Loss: {:.3f}, Accuracy: {:.3%}".format(epoch, epoch_loss_avg.result(), epoch_accuracy.result()))

    # # save model if new min for train loss is found
    # if epoch_loss_avg.result().numpy() < tmpTrain:
    tmpTrain = epoch_loss_avg.result().numpy()
    model.save_weights(checkpoint_path.format(epoch=epoch))

    epoch_loss_avg.reset_states()
    epoch_accuracy.reset_states()

### Output Training Details on Log File

In [ ]:
nameLogFile = 'log.txt'
logFile = open(save_path + nameLogFile, "w")

# write name of model
logFile.write("\n" + time + "\n\n")

# write hyper parameters
for i in range(len(listHyper0)):
    logFile.write(listHyper0[i] + ":  " + listHyper1[i] + "\n")

# write training details
logFile.write('\nTRAINING')
trainLossArr = np.asarray(train_loss_results)
trainAccArr = np.asarray(train_accuracy_results)
for i in range(numEpoTop+numEpoAll):
    logFile.write("\nEpoch {:03d}:   Loss: {:6.3f},   Accuracy: {:6.3%}".format(i+1, trainLossArr[i], trainAccArr[i]))

### Evaluate Models. Output Details on the logFile.

In [ ]:
### get the dataset

# name of dataset with sentences
data_name = "Scriber"
fileName = 'Data' + data_name + '/' + 'raw.processed.Test_01.txt'

# from sentences to list of words+punctuation
data = load_file(processingScriber00(fileName))

# encode and insert target
X_, y_ = encode_data(data, tokenizer, punctuation_enc)
X = insert_target(X_, sequenceSize)
y = np.asarray(y_)

# ### get only an n of the data.
# n = 32
# print(X.shape)
# X = X[0:n]
# y = y[0:n]
# print(X.shape)

# one hot encode the labels
y = tf.one_hot(y, 2, dtype='int64').numpy()

dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(batchSize)

In [ ]:
### build and compile model

bert_input = tf.keras.Input(shape=(sequenceSize), dtype='int32', name='bert_input')
x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")(bert_input)[0]
x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
dense_out = tf.keras.layers.Dense(2, activation='softmax')(x)

model = tf.keras.Model(bert_input, dense_out, name='model')

model.compile(optimizer='adam',
              loss=tf.losses.CategoricalCrossentropy(from_logits=False),
              metrics=[tf.keras.metrics.Recall(class_id=0, name='Rec_0'),
                       tf.keras.metrics.Precision(class_id=0, name='Prec_0'),
                       tf.keras.metrics.Recall(class_id=1, name='Rec_1'),
                       tf.keras.metrics.Precision(class_id=1, name='Prec_1'),
                      ])

In [ ]:
modelsLst = []
for r, d, f in os.walk(save_path):
    for file in sorted(f):
        if ".index" in file:
            modelsLst.append(file[:-6])

In [ ]:
# compute f1 score
def compF1(rec, pre):
    return 2 * (pre*rec) / (pre+rec)

In [ ]:
### evaluate models

print("\nEVALUATE")

print("\nEVALUATION DATASET TENSOR SHAPE = ", X.shape)

logFile.write('\n\nEVALUATION\n')
for i in range(len(modelsLst)):
    checkpointPath = save_path + modelsLst[i]
    print(checkpointPath)

    # load weights
    model.load_weights(checkpointPath)

    # evaluate
    evaluation = model.evaluate(dataset)
    
    f1_0 = compF1(evaluation[1],evaluation[2])
    f1_1 = compF1(evaluation[3],evaluation[4])
    print("F1_0 = {:9.6f} - F1_1 = {:9.6f}".format(f1_0, f1_1))
    
    # write details on log files
    logFile.write(modelsLst[i])
    logFile.write(" - Loss = {:7.4f} - Rec_0 = {:6.4f} - Pre_0 = {:6.4f} - F1_0 = {:9.6f} - Rec_1 = {:6.4f} - Pre_1 = {:6.4f} - F1_1 = {:9.6f}\n".format(evaluation[0], evaluation[1], evaluation[2], f1_0, evaluation[3], evaluation[4], f1_1))

logFile.close()